In [43]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

# Question 1: Install Spark and PySpark

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [6]:
spark.conf.set("spark.hadoop.io.nativeio", "false")

In [2]:
spark.version

'3.5.2'

---

# Question 2: Yellow October 2024


In [3]:
df = spark.read.parquet(r"C:\Users\karee\Downloads\yellow_tripdata_2024-10.parquet")

df.repartition(4)   \
    .write    \
    .parquet(r"K:\\Kaggle data sets\\yellow\\", mode="overwrite")

---

# Question 3: Count records 

In [4]:
df = spark.read.parquet(r"K:\\Kaggle data sets\\yellow\\*")

In [5]:
df.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-10 10:34:57|  2024-10-10 10:53:48|              1|          3.4|         1|                 N|         140|          74|           1|       20.5|  0.0|    0.5|      0.2

In [8]:
df_result = df.select("*").filter("tpep_pickup_datetime >= '2024-10-15 00:00:00' and tpep_pickup_datetime <= '2024-10-15 23:59:59'")

In [22]:
df_result.count()

128893

In [11]:
df.createTempView("yellow_taxi")

In [19]:
spark.sql("""
          SELECT
             COUNT(*)
          FROM
             yellow_taxi
          WHERE
             DATE(tpep_pickup_datetime) = '2024-10-15'
          """).show()

+--------+
|count(1)|
+--------+
|  128893|
+--------+



---

# Question 4: Longest trip

In [26]:
spark.sql("""
          SELECT
             MAX(datediff(HOUR, tpep_pickup_datetime, tpep_dropoff_datetime)) AS MAX_trip_duration
          FROM
             yellow_taxi
          """).show()

+-----------------+
|MAX_trip_duration|
+-----------------+
|              162|
+-----------------+



---

# Question 5: User Interface

4040

---

# Question 6: Least frequent pickup location zone

In [29]:
dff = spark.read.csv(r"C:/Users/karee/Downloads/taxi_zone_lookup.csv", header=True, inferSchema=True)

In [35]:
df.show(1)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-10 10:34:57|  2024-10-10 10:53:48|              1|          3.4|         1|                 N|         140|          74|           1|       20.5|  0.0|    0.5|      0.2

In [36]:
dff.show(1)

+----------+-------+--------------+------------+
|LocationID|Borough|          Zone|service_zone|
+----------+-------+--------------+------------+
|         1|    EWR|Newark Airport|         EWR|
+----------+-------+--------------+------------+
only showing top 1 row



In [32]:
dff.createTempView("taxi_zone_lookup")

In [ ]:
df_join = spark.sql(""" 
                    SELECT 
                        t.zone,
                        COUNT(1) AS frequency
                    FROM 
                        yellow_taxi y
                    JOIN taxi_zone_lookup t
                    ON y.PULocationID = t.LocationID
                    GROUP BY 
                        1
                    ORDER BY
                        frequency asc
                    """)
